<a href="https://colab.research.google.com/github/willaure/Analyzing-wildfire-activities-in-Australia_2/blob/main/Part_2_Creating_Dashboards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Get the tool ready
# Install python packages required to run the application. Copy and paste the below command to the terminal.

In [3]:
!pip3.8 install setuptools
!python3.8 -m pip install packaging
!python3.8 -m pip install pandas dash
!pip3 install httpx==0.20 dash plotly
!pip install --upgrade httpx anyio


/bin/bash: line 1: pip3.8: command not found
/bin/bash: line 1: python3.8: command not found
/bin/bash: line 1: python3.8: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.12.0
    Uninstalling h11-0.12.0:
      Successfully uninstalled h11-0.12.0
  Attempting uninstall: anyio
    Found existing installation: anyio 3.7.1
    Uninstalling anyio-3.7.1:
      Successfully uninstalled anyio-3.7.1
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.13.7
    Uninstalling httpcore-0.13.7:
      Successfully uninstalled httpcore-0.13.7
  Attempting uninstall: httpx
    Found existing installation: httpx 0.20.0
    Uninstallin

In [ ]:
# Let's create the application
#Create a new file called Dash_wildfire.py

In [4]:
code = '''
import dash
from dash import html

# Create a simple Dash app
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1("Wildfire Dashboard", style={'textAlign': 'center'}),
    html.P("This is a placeholder for the wildfire dashboard application.")
])

if __name__ == '__main__':
    app.run_server(debug=True)
'''

# Save to file
with open("Dash_wildfire.py", "w") as file:
    file.write(code)

print("✅ Dash_wildfire.py has been created!")


✅ Dash_wildfire.py has been created!


In [5]:
from google.colab import files
files.download("Dash_wildfire.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Get the application skeleton
#You can use this as a base code to complete the task below.
#Structure of the skeleton file

In [8]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt

# Create the app
app = dash.Dash(__name__)

# Clear the layout and do not display exceptions until the callback gets executed
app.config.suppress_callback_exceptions = True

# Read the wildfire data into a pandas dataframe
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # Used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year

# Layout Section of Dash

# Task 2.1: Add the Title to the Dashboard
app.layout = html.Div(children=[
    html.H1('Wildfire Data Analysis Dashboard'),

    # TASK 2.2: Add the radio items and a dropdown right below the first inner division
    html.Div([
        # First inner division for adding dropdown helper text for Selected Drive wheels
        html.Div([
            html.H2('Select Region'),

            # Radio items to select the region
            dcc.RadioItems([
                {"label": "New South Wales", "value": "NSW"},
                {"label": "Queensland", "value": "QL"},
                {"label": "South Australia", "value": "SA"},
                {"label": "Tasmania", "value": "TA"},
                {"label": "Victoria", "value": "VI"},
                {"label": "Western Australia", "value": "WA"},
                {"label": "Northern Territory", "value": "NT"}
            ], value="NSW", id='region-selector', inline=True),

            # Dropdown to select year
            html.Div([
                html.H2('Select Year', style={'padding-top': '10px'}),
                dcc.Dropdown(
                    id='year-selector',
                    options=[{'label': str(year), 'value': year} for year in sorted(df['Year'].unique())],
                    value=df['Year'].max(),  # default value is the latest year
                )
            ])
        ]),

        # Second inner division for adding 2 inner divisions for 2 output graphs
        html.Div([
            html.Div([ ], id='graph-1'),
            html.Div([ ], id='graph-2')
        ], style={'display': 'flex'}),

    ])
])

# TASK 2.4: Add the Output and input components inside the app.callback decorator.
@app.callback(
    [Output(component_id='graph-1', component_property='children'),
     Output(component_id='graph-2', component_property='children')],
    [Input(component_id='region-selector', component_property='value'),
     Input(component_id='year-selector', component_property='value')]
)
# TASK 2.5: Add the callback function.
def reg_year_display(input_region, input_year):
    # Filter data based on region and year
    region_data = df[df['Region'] == input_region]
    year_data = region_data[region_data['Year'] == input_year]

    # Plot one - Monthly Average Estimated Fire Area
    est_data = year_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(est_data, names='Month', values='Estimated_fire_area', title="{} : Monthly Average Estimated Fire Area in year {}".format(input_region, input_year))

    # Plot two - Monthly Average Count of Pixels for Presumed Vegetation Fires
    veg_data = year_data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(veg_data, x='Month', y='Count', title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(input_region, input_year))

    return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)]


if __name__ == '__main__':
    app.run(debug=True)  # Note: Corrected to app.run instead of app.run_server



<IPython.core.display.Javascript object>

In [ ]:
#TASK 2.1: Add title to the dashboard
#Update the html.H1() tag to hold the application title.

#Application title is Australia Wildfire Dashboard
#Use style parameter provided below to make the title center aligned, with color code #503D36, and font-size as 26

In [10]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt

# Create the app
app = dash.Dash(__name__)

# Clear the layout and do not display exceptions until the callback gets executed
app.config.suppress_callback_exceptions = True

# Read the wildfire data into a pandas dataframe
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # Used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year

# Layout Section of Dash

# Task 2.1: Add the Title to the Dashboard
app.layout = html.Div(children=[
    html.H1(
        'Australia Wildfire Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}
    ),

    # TASK 2.2: Add the radio items and a dropdown right below the first inner division
    html.Div([
        # First inner division for adding dropdown helper text for Selected Drive wheels
        html.Div([
            html.H2('Select Region'),

            # Radio items to select the region
            dcc.RadioItems([
                {"label": "New South Wales", "value": "NSW"},
                {"label": "Queensland", "value": "QL"},
                {"label": "South Australia", "value": "SA"},
                {"label": "Tasmania", "value": "TA"},
                {"label": "Victoria", "value": "VI"},
                {"label": "Western Australia", "value": "WA"},
                {"label": "Northern Territory", "value": "NT"}
            ], value="NSW", id='region-selector', inline=True),

            # Dropdown to select year
            html.Div([
                html.H2('Select Year', style={'padding-top': '10px'}),
                dcc.Dropdown(
                    id='year-selector',
                    options=[{'label': str(year), 'value': year} for year in sorted(df['Year'].unique())],
                    value=df['Year'].max(),  # default value is the latest year
                )
            ])
        ]),

        # Second inner division for adding 2 inner divisions for 2 output graphs
        html.Div([
            html.Div([ ], id='graph-1'),
            html.Div([ ], id='graph-2')
        ], style={'display': 'flex'}),

    ])
])

# TASK 2.4: Add the Output and input components inside the app.callback decorator.
@app.callback(
    [Output(component_id='graph-1', component_property='children'),
     Output(component_id='graph-2', component_property='children')],
    [Input(component_id='region-selector', component_property='value'),
     Input(component_id='year-selector', component_property='value')]
)
# TASK 2.5: Add the callback function.
def reg_year_display(input_region, input_year):
    # Filter data based on region and year
    region_data = df[df['Region'] == input_region]
    year_data = region_data[region_data['Year'] == input_year]

    # Plot one - Monthly Average Estimated Fire Area
    est_data = year_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(est_data, names='Month', values='Estimated_fire_area', title="{} : Monthly Average Estimated Fire Area in year {}".format(input_region, input_year))

    # Plot two - Monthly Average Count of Pixels for Presumed Vegetation Fires
    veg_data = year_data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(veg_data, x='Month', y='Count', title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(input_region, input_year))

    return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)]


if __name__ == '__main__':
    app.run(debug=True)  # Corrected to app.run instead of app.run_server


<IPython.core.display.Javascript object>

In [ ]:
# TASK 2.2: Add the radio items and a dropdown right below the first inner division.
# Radio items to choose the Region
#
# The radio items work similar to the dropdown, you need to call dcc.RadioItems and pass the list of items.
# Make use of inline=True property to display the radio items in a horizontal line
#
# You can extract the regions from the dataframe using df.Region.unique() or pass the list of all regions directly as ['NSW','QL','SA','TA','VI','WA','NT'] .
# Assign radioitems id as region
# Label as Select Region
# value as NSW
# For your reference below are the abbreviations used in the dataset for regions
# NSW - New South Wales
# NT - Northern Territory
# QL - Queensland
# SA - South Australia
# TA - Tasmania
# VI - Victoria
# WA - Western Australia
# Read more on RadioItems

In [11]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt

# Create the app
app = dash.Dash(__name__)

# Clear the layout and do not display exceptions until the callback gets executed
app.config.suppress_callback_exceptions = True

# Read the wildfire data into a pandas dataframe
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # Used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year

# Layout Section of Dash

# Task 2.1: Add the Title to the Dashboard
app.layout = html.Div(children=[
    html.H1(
        'Australia Wildfire Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}
    ),

    # TASK 2.2: Add the radio items and a dropdown right below the first inner division
    html.Div([
        # First inner division for adding dropdown helper text for Selected Drive wheels
        html.Div([
            html.H2('Select Region'),

            # Radio items to select the region
            dcc.RadioItems(
                options=[
                    {"label": "New South Wales", "value": "NSW"},
                    {"label": "Queensland", "value": "QL"},
                    {"label": "South Australia", "value": "SA"},
                    {"label": "Tasmania", "value": "TA"},
                    {"label": "Victoria", "value": "VI"},
                    {"label": "Western Australia", "value": "WA"},
                    {"label": "Northern Territory", "value": "NT"}
                ],
                value="NSW",  # default value
                id='region',
                inline=True  # Display radio items inline
            ),

            # Dropdown to select year
            html.Div([
                html.H2('Select Year', style={'padding-top': '10px'}),
                dcc.Dropdown(
                    id='year-selector',
                    options=[{'label': str(year), 'value': year} for year in sorted(df['Year'].unique())],
                    value=df['Year'].max(),  # default value is the latest year
                )
            ])
        ]),

        # Second inner division for adding 2 inner divisions for 2 output graphs
        html.Div([
            html.Div([ ], id='graph-1'),
            html.Div([ ], id='graph-2')
        ], style={'display': 'flex'}),

    ])
])

# TASK 2.4: Add the Output and input components inside the app.callback decorator.
@app.callback(
    [Output(component_id='graph-1', component_property='children'),
     Output(component_id='graph-2', component_property='children')],
    [Input(component_id='region', component_property='value'),
     Input(component_id='year-selector', component_property='value')]
)
# TASK 2.5: Add the callback function.
def reg_year_display(input_region, input_year):
    # Filter data based on region and year
    region_data = df[df['Region'] == input_region]
    year_data = region_data[region_data['Year'] == input_year]

    # Plot one - Monthly Average Estimated Fire Area
    est_data = year_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(est_data, names='Month', values='Estimated_fire_area', title="{} : Monthly Average Estimated Fire Area in year {}".format(input_region, input_year))

    # Plot two - Monthly Average Count of Pixels for Presumed Vegetation Fires
    veg_data = year_data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(veg_data, x='Month', y='Count', title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(input_region, input_year))

    return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)]


if __name__ == '__main__':
    app.run(debug=True)  # Corrected to app.run instead of app.run_server


<IPython.core.display.Javascript object>

In [12]:
# Dropdown to choose the Year
# The dropdown has an id as year.
# The label as Select Year
# The values allowed in the dropdown are years from 2005 to 2020
# The default value when the dropdown is displayed is 2005.

In [13]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt

# Create the app
app = dash.Dash(__name__)

# Clear the layout and do not display exceptions until the callback gets executed
app.config.suppress_callback_exceptions = True

# Read the wildfire data into a pandas dataframe
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # Used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year

# Layout Section of Dash
app.layout = html.Div(children=[
    html.H1(
        'Australia Wildfire Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}
    ),

    # TASK 2.2: Add the radio items and a dropdown right below the first inner division
    html.Div([
        # First inner division for adding dropdown helper text for Selected Drive wheels
        html.Div([
            html.H2('Select Region'),

            # Radio items to select the region
            dcc.RadioItems(
                options=[
                    {"label": "New South Wales", "value": "NSW"},
                    {"label": "Queensland", "value": "QL"},
                    {"label": "South Australia", "value": "SA"},
                    {"label": "Tasmania", "value": "TA"},
                    {"label": "Victoria", "value": "VI"},
                    {"label": "Western Australia", "value": "WA"},
                    {"label": "Northern Territory", "value": "NT"}
                ],
                value="NSW",  # default value
                id='region',
                inline=True  # Display radio items inline
            ),

            # Dropdown to select year
            html.Div([
                html.H2('Select Year', style={'padding-top': '10px'}),
                dcc.Dropdown(
                    id='year',
                    options=[{'label': str(year), 'value': year} for year in range(2005, 2021)],  # Year range from 2005 to 2020
                    value=2005,  # default value is 2005
                )
            ])
        ]),

        # Second inner division for adding 2 inner divisions for 2 output graphs
        html.Div([
            html.Div([ ], id='graph-1'),
            html.Div([ ], id='graph-2')
        ], style={'display': 'flex'}),

    ])
])

# TASK 2.4: Add the Output and input components inside the app.callback decorator.
@app.callback(
    [Output(component_id='graph-1', component_property='children'),
     Output(component_id='graph-2', component_property='children')],
    [Input(component_id='region', component_property='value'),
     Input(component_id='year', component_property='value')]
)
# TASK 2.5: Add the callback function.
def reg_year_display(input_region, input_year):
    # Filter data based on region and year
    region_data = df[df['Region'] == input_region]
    year_data = region_data[region_data['Year'] == input_year]

    # Plot one - Monthly Average Estimated Fire Area
    est_data = year_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(est_data, names='Month', values='Estimated_fire_area', title="{} : Monthly Average Estimated Fire Area in year {}".format(input_region, input_year))

    # Plot two - Monthly Average Count of Pixels for Presumed Vegetation Fires
    veg_data = year_data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(veg_data, x='Month', y='Count', title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(input_region, input_year))

    return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)]


if __name__ == '__main__':
    app.run(debug=True)  # Corrected to app.run instead of app.run_server


<IPython.core.display.Javascript object>

In [ ]:
#TASK 2.3: Add two empty divisions for output inside the next inner division.
#Use 2 html.Div() tags .
#Provide division ids as plot1 and plot2.

In [14]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt

# Create the app
app = dash.Dash(__name__)

# Clear the layout and do not display exceptions until the callback gets executed
app.config.suppress_callback_exceptions = True

# Read the wildfire data into a pandas dataframe
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # Used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year

# Layout Section of Dash
app.layout = html.Div(children=[
    html.H1(
        'Australia Wildfire Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}
    ),

    # TASK 2.2: Add the radio items and a dropdown right below the first inner division
    html.Div([
        # First inner division for adding dropdown helper text for Selected Drive wheels
        html.Div([
            html.H2('Select Region'),

            # Radio items to select the region
            dcc.RadioItems(
                options=[
                    {"label": "New South Wales", "value": "NSW"},
                    {"label": "Queensland", "value": "QL"},
                    {"label": "South Australia", "value": "SA"},
                    {"label": "Tasmania", "value": "TA"},
                    {"label": "Victoria", "value": "VI"},
                    {"label": "Western Australia", "value": "WA"},
                    {"label": "Northern Territory", "value": "NT"}
                ],
                value="NSW",  # default value
                id='region',
                inline=True  # Display radio items inline
            ),

            # Dropdown to select year
            html.Div([
                html.H2('Select Year', style={'padding-top': '10px'}),
                dcc.Dropdown(
                    id='year',
                    options=[{'label': str(year), 'value': year} for year in range(2005, 2021)],  # Year range from 2005 to 2020
                    value=2005,  # default value is 2005
                )
            ])
        ]),

        # Second inner division for adding 2 inner divisions for 2 output graphs (TASK 2.3)
        html.Div([
            html.Div([ ], id='plot1'),  # First empty division for plot1
            html.Div([ ], id='plot2')   # Second empty division for plot2
        ], style={'display': 'flex'})  # To align the plots side by side

    ])
])

# TASK 2.4: Add the Output and input components inside the app.callback decorator.
@app.callback(
    [Output(component_id='plot1', component_property='children'),
     Output(component_id='plot2', component_property='children')],
    [Input(component_id='region', component_property='value'),
     Input(component_id='year', component_property='value')]
)
# TASK 2.5: Add the callback function.
def reg_year_display(input_region, input_year):
    # Filter data based on region and year
    region_data = df[df['Region'] == input_region]
    year_data = region_data[region_data['Year'] == input_year]

    # Plot one - Monthly Average Estimated Fire Area
    est_data = year_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(est_data, names='Month', values='Estimated_fire_area', title="{} : Monthly Average Estimated Fire Area in year {}".format(input_region, input_year))

    # Plot two - Monthly Average Count of Pixels for Presumed Vegetation Fires
    veg_data = year_data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(veg_data, x='Month', y='Count', title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(input_region, input_year))

    return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)]


if __name__ == '__main__':
    app.run(debug=True)  # Corrected to app.run instead of app.run_server


<IPython.core.display.Javascript object>

In [15]:
# TASK 2.4: Add the Output and Input components inside the app.callback decorator.
# The inputs and outputs of our application’s interface are described declaratively as the arguments of @app.callback decorator.
# - In Dash, the inputs and outputs of our application are simply the properties of a particular component.
#
# In this example, we have two inputs:
# - input for Region is the value property of the component that has the ID region
# - input for Year is the value property of the component that has the ID year
#
# Our layout has 2 outputs so we need to create 2 output components.
# It is a list with 2 output parameters with component id and property. Here, the component property will be children as we have created empty division and passing in dcc.Graph (figure) after computation.
#
# Component ids will be plot1 , plot2.

In [16]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt

# Create the app
app = dash.Dash(__name__)

# Clear the layout and do not display exceptions until the callback gets executed
app.config.suppress_callback_exceptions = True

# Read the wildfire data into a pandas dataframe
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # Used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year

# Layout Section of Dash
app.layout = html.Div(children=[
    html.H1(
        'Australia Wildfire Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}
    ),

    # TASK 2.2: Add the radio items and a dropdown right below the first inner division
    html.Div([
        # First inner division for adding dropdown helper text for Selected Drive wheels
        html.Div([
            html.H2('Select Region'),

            # Radio items to select the region
            dcc.RadioItems(
                options=[
                    {"label": "New South Wales", "value": "NSW"},
                    {"label": "Queensland", "value": "QL"},
                    {"label": "South Australia", "value": "SA"},
                    {"label": "Tasmania", "value": "TA"},
                    {"label": "Victoria", "value": "VI"},
                    {"label": "Western Australia", "value": "WA"},
                    {"label": "Northern Territory", "value": "NT"}
                ],
                value="NSW",  # default value
                id='region',
                inline=True  # Display radio items inline
            ),

            # Dropdown to select year
            html.Div([
                html.H2('Select Year', style={'padding-top': '10px'}),
                dcc.Dropdown(
                    id='year',
                    options=[{'label': str(year), 'value': year} for year in range(2005, 2021)],  # Year range from 2005 to 2020
                    value=2005,  # default value is 2005
                )
            ])
        ]),

        # Second inner division for adding 2 inner divisions for 2 output graphs (TASK 2.3)
        html.Div([
            html.Div([ ], id='plot1'),  # First empty division for plot1
            html.Div([ ], id='plot2')   # Second empty division for plot2
        ], style={'display': 'flex'})  # To align the plots side by side

    ])
])

# TASK 2.4: Add the Output and input components inside the app.callback decorator.
@app.callback(
    [Output(component_id='plot1', component_property='children'),
     Output(component_id='plot2', component_property='children')],
    [Input(component_id='region', component_property='value'),
     Input(component_id='year', component_property='value')]
)
# TASK 2.5: Add the callback function.
def reg_year_display(input_region, input_year):
    # Filter data based on region and year
    region_data = df[df['Region'] == input_region]
    year_data = region_data[region_data['Year'] == input_year]

    # Plot one - Monthly Average Estimated Fire Area
    est_data = year_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(est_data, names='Month', values='Estimated_fire_area', title="{} : Monthly Average Estimated Fire Area in year {}".format(input_region, input_year))

    # Plot two - Monthly Average Count of Pixels for Presumed Vegetation Fires
    veg_data = year_data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(veg_data, x='Month', y='Count', title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(input_region, input_year))

    return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)]


if __name__ == '__main__':
    app.run(debug=True)  # Corrected to app.run instead of app.run_server


<IPython.core.display.Javascript object>

In [17]:
# TASK 2.5: Add the callback function.
# Whenever an input property changes, the function that the callback decorator wraps will get called automatically.
# In this case, let us define a function reg_year_display() which will be wrapped by our decorator.
#
# The function first filters our dataframe df by the selected value of the region from the radio items and year from the dropdown as follows:
# region_data = df[df['Region'] == input_region]
# y_r_data = region_data[region_data['Year'] == input_year]
#
# For pie chart on Monthly Average Estimated Fire Area:
# Next we will group by the Month and calculate the mean Estimated_fire_area of the dataframe.
# Use the px.pie() function to plot the pie chart.
#
# For bar chart on Monthly Average Count of Pixels for Presumed Vegetation Fires:
# Next we will group by the Month and calculate the mean Count of the dataframe.
# Use the px.bar() function to plot the bar chart.

In [18]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt

# Create the app
app = dash.Dash(__name__)

# Clear the layout and do not display exceptions until the callback gets executed
app.config.suppress_callback_exceptions = True

# Read the wildfire data into a pandas dataframe
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # Used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year

# Layout Section of Dash
app.layout = html.Div(children=[
    html.H1(
        'Australia Wildfire Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}
    ),

    # TASK 2.2: Add the radio items and a dropdown right below the first inner division
    html.Div([
        # First inner division for adding dropdown helper text for Selected Drive wheels
        html.Div([
            html.H2('Select Region'),

            # Radio items to select the region
            dcc.RadioItems(
                options=[
                    {"label": "New South Wales", "value": "NSW"},
                    {"label": "Queensland", "value": "QL"},
                    {"label": "South Australia", "value": "SA"},
                    {"label": "Tasmania", "value": "TA"},
                    {"label": "Victoria", "value": "VI"},
                    {"label": "Western Australia", "value": "WA"},
                    {"label": "Northern Territory", "value": "NT"}
                ],
                value="NSW",  # default value
                id='region',
                inline=True  # Display radio items inline
            ),

            # Dropdown to select year
            html.Div([
                html.H2('Select Year', style={'padding-top': '10px'}),
                dcc.Dropdown(
                    id='year',
                    options=[{'label': str(year), 'value': year} for year in range(2005, 2021)],  # Year range from 2005 to 2020
                    value=2005,  # default value is 2005
                )
            ])
        ]),

        # Second inner division for adding 2 inner divisions for 2 output graphs (TASK 2.3)
        html.Div([
            html.Div([ ], id='plot1'),  # First empty division for plot1
            html.Div([ ], id='plot2')   # Second empty division for plot2
        ], style={'display': 'flex'})  # To align the plots side by side

    ])
])

# TASK 2.4: Add the Output and input components inside the app.callback decorator.
@app.callback(
    [Output(component_id='plot1', component_property='children'),
     Output(component_id='plot2', component_property='children')],
    [Input(component_id='region', component_property='value'),
     Input(component_id='year', component_property='value')]
)
# TASK 2.5: Add the callback function.
def reg_year_display(input_region, input_year):
    # Filter data based on region and year
    region_data = df[df['Region'] == input_region]
    year_data = region_data[region_data['Year'] == input_year]

    # Pie chart for Monthly Average Estimated Fire Area
    est_data = year_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(est_data, names='Month', values='Estimated_fire_area',
                  title="{} : Monthly Average Estimated Fire Area in year {}".format(input_region, input_year))

    # Bar chart for Monthly Average Count of Pixels for Presumed Vegetation Fires
    veg_data = year_data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(veg_data, x='Month', y='Count',
                  title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(input_region, input_year))

    # Return the figures as graphs to be displayed in the output divisions
    return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)]


if __name__ == '__main__':
    app.run(debug=True)  # Corrected to app.run instead of app.run_server


<IPython.core.display.Javascript object>

In [ ]:
#Run the Application
#Next Run the python file using the command

In [19]:
!pip install dash jupyter-dash plotly pandas
import pandas as pd
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
# Load the wildfire dataset
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Process the data
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # Extract the month name
df['Year'] = pd.to_datetime(df['Date']).dt.year  # Extract the year


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00


In [20]:
# Create Dash app
app = JupyterDash(__name__)

# Application layout
app.layout = html.Div([
    html.H1('Australia Wildfire Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}),

    # Region selection (RadioItems)
    html.Div([
        html.H2('Select Region'),
        dcc.RadioItems(
            id='region',
            options=[
                {'label': 'New South Wales', 'value': 'NSW'},
                {'label': 'Northern Territory', 'value': 'NT'},
                {'label': 'Queensland', 'value': 'QL'},
                {'label': 'South Australia', 'value': 'SA'},
                {'label': 'Tasmania', 'value': 'TA'},
                {'label': 'Victoria', 'value': 'VI'},
                {'label': 'Western Australia', 'value': 'WA'}
            ],
            value='NSW',
            inline=True
        ),

        # Year selection (Dropdown)
        html.H2('Select Year'),
        dcc.Dropdown(
            id='year',
            options=[{'label': str(year), 'value': year} for year in range(2005, 2021)],
            value=2005
        )
    ]),

    # Output plots (empty divisions for graphs)
    html.Div([
        html.Div(id='plot1', style={'display': 'inline-block'}),
        html.Div(id='plot2', style={'display': 'inline-block'})
    ])
])


/usr/local/lib/python3.11/dist-packages/dash/dash.py:582: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [21]:
@app.callback(
    [Output('plot1', 'children'),
     Output('plot2', 'children')],
    [Input('region', 'value'),
     Input('year', 'value')]
)
def reg_year_display(input_region, input_year):
    # Filter data based on selected region and year
    region_data = df[df['Region'] == input_region]
    year_data = region_data[region_data['Year'] == input_year]

    # Pie chart for monthly average estimated fire area
    fire_area_avg = year_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(fire_area_avg, names='Month', values='Estimated_fire_area',
                  title=f"{input_region} : Monthly Average Estimated Fire Area in {input_year}")

    # Bar chart for monthly average count of pixels for presumed vegetation fires
    veg_fire_avg = year_data.groupby('Month')['Count_of_pixels'].mean().reset_index()
    fig2 = px.bar(veg_fire_avg, x='Month', y='Count_of_pixels',
                  title=f"{input_region} : Average Count of Pixels for Presumed Vegetation Fires in {input_year}")

    return dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)


In [25]:
# Install necessary libraries
!pip install dash plotly pandas

# Import necessary libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Load the wildfire dataset
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Process the data
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # Extract the month name
df['Year'] = pd.to_datetime(df['Date']).dt.year  # Extract the year

# Create Dash app
app = dash.Dash(__name__)

# Application layout
app.layout = html.Div([
    html.H1('Australia Wildfire Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 26}),

    # Region selection (RadioItems)
    html.Div([
        html.H2('Select Region'),
        dcc.RadioItems(
            id='region',
            options=[
                {'label': 'New South Wales', 'value': 'NSW'},
                {'label': 'Northern Territory', 'value': 'NT'},
                {'label': 'Queensland', 'value': 'QL'},
                {'label': 'South Australia', 'value': 'SA'},
                {'label': 'Tasmania', 'value': 'TA'},
                {'label': 'Victoria', 'value': 'VI'},
                {'label': 'Western Australia', 'value': 'WA'}
            ],
            value='NSW',
            inline=True
        ),

        # Year selection (Dropdown)
        html.H2('Select Year'),
        dcc.Dropdown(
            id='year',
            options=[{'label': str(year), 'value': year} for year in range(2005, 2021)],
            value=2005
        )
    ]),

    # Output plots (empty divisions for graphs)
    html.Div([
        html.Div(id='plot1', style={'display': 'inline-block'}),
        html.Div(id='plot2', style={'display': 'inline-block'})
    ])
])

# Define callback function for updates
@app.callback(
    [Output('plot1', 'children'),
     Output('plot2', 'children')],
    [Input('region', 'value'),
     Input('year', 'value')]
)
def reg_year_display(input_region, input_year):
    # Filter data based on selected region and year
    region_data = df[df['Region'] == input_region]
    year_data = region_data[region_data['Year'] == input_year]

    # Pie chart for monthly average estimated fire area
    fire_area_avg = year_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(fire_area_avg, names='Month', values='Estimated_fire_area',
                  title=f"{input_region} : Monthly Average Estimated Fire Area in {input_year}")

    # Bar chart for monthly average count of pixels for presumed vegetation fires
    veg_fire_avg = year_data.groupby('Month')['Count_of_pixels'].mean().reset_index()
    fig2 = px.bar(veg_fire_avg, x='Month', y='Count_of_pixels',
                  title=f"{input_region} : Average Count of Pixels for Presumed Vegetation Fires in {input_year}")

    return dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)

# Run the Dash app
app.run(debug=True, use_reloader=False, port=8050)


<IPython.core.display.Javascript object>